<a href="https://colab.research.google.com/github/Luz-Aquino/chatbot-supermercado-ia/blob/main/Codigo_Supermercado_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio thefuzz groq

import sqlite3
import pandas as pd
from thefuzz import process
import gradio as gr
from groq import Groq
import re
from datetime import datetime

# --- 🔑 API KEY ---
API_KEY = ""

try:
    client = Groq(api_key=API_KEY)
except:
    pass

# --- 1. BASE DE DATOS ---
conn = sqlite3.connect(':memory:', check_same_thread=False)
cursor = conn.cursor()

cursor.execute('DROP TABLE IF EXISTS usuarios')
cursor.execute('''CREATE TABLE usuarios (dni TEXT PRIMARY KEY, password TEXT, nombre TEXT, apellido TEXT, mail TEXT, fecha_nac TEXT, direccion TEXT)''')
cursor.execute('DROP TABLE IF EXISTS productos')
cursor.execute('''CREATE TABLE productos (id INTEGER PRIMARY KEY, nombre TEXT, marca TEXT, categoria TEXT, precio REAL, stock INTEGER, unidad_medida TEXT, promocion TEXT)''')

datos_super = [
    (1, 'Leche Entera', 'La Serenísima', 'Lacteos', 1200.0, 50, '1 Litro', 'Llevando 3 pagas 2'),
    (2, 'Arroz Oro', 'Gallo', 'Almacen', 1850.5, 100, '1 kg', None),
    (3, 'Gaseosa Cola Original', 'Coca Cola', 'Bebidas', 2400.0, 40, '2.25 L', '2da al 50%'),
    (4, 'Fernet', 'Branca', 'Bebidas', 9500.0, 5, '750 ml', '10% OFF con App'),
    (5, 'Jabón Liquido', 'Ariel', 'Limpieza', 8200.0, 20, '3 L', None),
    (6, 'Yerba Mate', 'Playadito', 'Almacen', 3800.0, 60, '1 kg', 'Precio Cuidado'),
    (7, 'Fideos Tirabuzón', 'Matarazzo', 'Almacen', 1100.0, 80, '500 g', '3x2'),
    (8, 'Aceite Girasol', 'Natura', 'Almacen', 2100.0, 45, '1.5 L', None),
    (9, 'Azúcar Clásica', 'Ledesma', 'Almacen', 950.0, 90, '1 kg', None),
    (10, 'Harina 0000', 'Pureza', 'Almacen', 850.0, 70, '1 kg', None),
    (11, 'Puré de Tomate', 'Arcor', 'Almacen', 700.0, 100, '520 g', '4x3'),
    (12, 'Galletitas Rellenas', 'Oreo', 'Almacen', 1200.0, 150, '117 g', '2x1'),
    (13, 'Queso Cremoso', 'Cremón', 'Lacteos', 5500.0, 30, 'Por kg', None),
    (14, 'Manteca', 'La Paulina', 'Lacteos', 2200.0, 40, '200 g', None),
    (15, 'Yogur Bebible', 'Ser', 'Lacteos', 1500.0, 35, '1 kg', '20% OFF'),
    (16, 'Dulce de Leche', 'La Colonial', 'Almacen', 2800.0, 50, '400 g', None),
    (17, 'Cerveza Rubia', 'Quilmes', 'Bebidas', 1800.0, 120, '1 L', 'Pack x6 $9000'),
    (18, 'Vino Malbec', 'Rutini', 'Bebidas', 12500.0, 10, '750 ml', '15% OFF Caja'),
    (19, 'Agua Mineral', 'Villavicencio', 'Bebidas', 900.0, 200, '1.5 L', None),
    (20, 'Jugo Naranja', 'Citric', 'Bebidas', 2600.0, 25, '1 L', None),
    (21, 'Lavandina', 'Ayudin', 'Limpieza', 1200.0, 60, '1 L', None),
    (22, 'Detergente', 'Magistral', 'Limpieza', 2300.0, 50, '500 ml', '2x1'),
    (23, 'Papel Higiénico', 'Higienol', 'Limpieza', 3500.0, 40, '4 rollos', None),
    (24, 'Shampoo', 'Pantene', 'Perfumeria', 4200.0, 30, '400 ml', '25% OFF 2da unidad'),
    (25, 'Desodorante', 'Dove', 'Perfumeria', 2800.0, 45, '150 ml', None),
    (26, 'Pan Lactal', 'Artesano', 'Panaderia', 3200.0, 25, '500 g', None),
    (27, 'Huevos Blancos', 'Avicola', 'Frescos', 2800.0, 40, '12 un', 'Precio Amigo'),
    (28, 'Pechuga de Pollo', 'Granja Tres Arroyos', 'Carniceria', 6500.0, 20, 'Por kg', None),
    (29, 'Carne Picada', 'Frigorífico', 'Carniceria', 7200.0, 15, 'Por kg', None),
    (30, 'Manzanas Rojas', 'Moño Azul', 'Verduleria', 1800.0, 30, 'Por kg', 'Oferta de Estación')
]
cursor.executemany('INSERT INTO productos VALUES (?,?,?,?,?,?,?,?)', datos_super)
cursor.execute("INSERT INTO usuarios VALUES ('12345678', 'Luz12345', 'Luz', 'Aquino', 'luz@test.com', '01/01/1990', 'Calle Falsa 123')")
conn.commit()

# --- HELPERS GENERALES ---
def formato_ar(valor):
    try:
        val = float(valor)
        return f"{val:,.2f}".replace(",", "X").replace(".", ",").replace("X", ".")
    except: return str(valor)

def obtener_categorias():
    df = pd.read_sql_query("SELECT DISTINCT categoria FROM productos", conn)
    return ["🔥 Ofertas del Día", "📂 Ver Todo"] + sorted(df['categoria'].tolist())

# --- LÓGICA PRECIOS (PARA USAR EN CHAT Y CARRITO) ---
def calcular_precio_marketing(precio, promo):
    """Devuelve un texto marketinero con la cuenta hecha"""
    if not promo: return ""
    promo = promo.lower()

    if "3 pagas 2" in promo or "3x2" in promo:
        total = precio * 2
        return f"Llevando 3 pagás solo 2: Total ${formato_ar(total)}."
    elif "2x1" in promo:
        total = precio
        return f"Llevando 2 pagás solo 1: Total ${formato_ar(total)}."
    elif "2da al 50" in promo:
        total = precio + (precio * 0.5)
        return f"Llevando 2, el total te queda en ${formato_ar(total)}."
    elif "% off" in promo and "caja" not in promo:
        nums = re.findall(r'\d+', promo)
        if nums:
            desc = int(nums[0])
            final = precio * ((100-desc)/100)
            return f"Con el {desc}% te queda en ${formato_ar(final)} por unidad."
    return f"Aprovechá la oferta: {promo}."

# --- BÚSQUEDA INTELIGENTE ---
def buscar_para_chat(mensaje):
    df = pd.read_sql_query("SELECT * FROM productos", conn)
    mensaje = mensaje.lower()

    # 1. BÚSQUEDA POR NÚMEROS (INDICES/IDS)
    numeros = re.findall(r'\b\d+\b', mensaje)
    ids_encontrados = []

    if numeros:
        for num in numeros:
            prod = df[df['id'] == int(num)]
            if not prod.empty:
                ids_encontrados.append(prod.iloc[0])

        if ids_encontrados:
            return "PRODUCTOS_MULTIPLES", pd.DataFrame(ids_encontrados)

    # 2. BÚSQUEDA POR NOMBRES MÚLTIPLES
    palabras_clave = re.split(r',| y | e ', mensaje) # Separa por coma o " y "
    productos_encontrados = []

    for palabra in palabras_clave:
        palabra = palabra.strip()
        if len(palabra) > 2: # Ignorar palabras cortas
            matches = df[df['nombre'].str.lower().str.contains(palabra) | df['marca'].str.lower().str.contains(palabra)]
            if not matches.empty:
                productos_encontrados.append(matches.iloc[0])

    if productos_encontrados:
        # Eliminamos duplicados
        df_final = pd.DataFrame(productos_encontrados).drop_duplicates()
        return "PRODUCTOS_MULTIPLES", df_final

    # 3. OFERTAS
    if "oferta" in mensaje or "descuento" in mensaje or "promo" in mensaje:
        return "LISTA_OFERTAS", df[df['promocion'].notnull()]

    # 4. CATEGORÍA
    categorias = df['categoria'].unique()
    for cat in categorias:
        if cat.lower() in mensaje:
            return "LISTA_CATEGORIA", df[df['categoria'] == cat]

    return "NADA", None

def responder_chat(mensaje):
    try:
        tipo, data = buscar_para_chat(mensaje)
        client = Groq(api_key=API_KEY)

        if tipo == "LISTA_OFERTAS":
            items = []
            for i, row in data.iterrows():
                items.append(f"{row['id']}. {row['nombre']} {row['marca']} (${formato_ar(row['precio'])}) - {row['promocion']}")
            texto_datos = "\n".join(items)

            prompt = f"""
            Usuario pide ofertas.
            Lista:
            {texto_datos}

            Tarea:
            1. Respondé: "¡Estas son las ofertas destacadas del día!"
            2. Listalas usando el formato: "1. NombreProducto ($Precio) - Oferta"
            3. Al final preguntá: "¿Te interesa alguno? Decime el número o nombre y te digo dónde encontrarlo."
            """

        elif tipo == "PRODUCTOS_MULTIPLES" or tipo == "PRODUCTO":
            if isinstance(data, pd.Series): data = pd.DataFrame([data])

            detalles = []
            for _, row in data.iterrows():
                marketing = calcular_precio_marketing(row['precio'], row['promocion'])
                if marketing:
                    linea = f"Tengo una oferta especial para ti: {row['promocion']} en {row['nombre']} {row['marca']}. {marketing} Lo encontrás en la categoría **{row['categoria']}**."
                else:
                    linea = f"{row['nombre']} {row['marca']} está a ${formato_ar(row['precio'])}. Lo encontrás en la categoría **{row['categoria']}**."
                detalles.append(linea)

            texto_final = "\n\n".join(detalles)

            prompt = f"""
            Usuario preguntó por productos específicos.
            Tenés esta información PRE-CALCULADA:

            {texto_final}

            Tarea:
            1. Repetí la información tal cual te la pasé arriba. No inventes precios ni cálculos.
            2. Cerrá preguntando: "¿Te gustaría consultar algo más?"
            """

        elif tipo == "LISTA_CATEGORIA":
            cat = data.iloc[0]['categoria']
            prompt = f"El usuario quiere ver '{cat}'. Decile que seleccione la categoría **{cat}** en el menú desplegable de la derecha para ver todo el catálogo."

        else:
            prompt = f"No entendiste '{mensaje}'. Decile que pruebe buscando un producto por nombre (ej: 'Coca y Fernet') o escribiendo 'Ofertas'."

        chat = client.chat.completions.create(
            messages=[{"role": "user", "content": prompt}],
            model="llama-3.3-70b-versatile"
        )
        return chat.choices[0].message.content

    except Exception as e:
        return f"⚠️ Error: {str(e)}"

# --- RESTO DEL SISTEMA (CARRITO) ---
def filtrar_por_categoria(cat):
    if not cat: return gr.update(choices=[])
    if cat == "🔥 Ofertas del Día": query = "SELECT nombre, marca, precio FROM productos WHERE promocion IS NOT NULL"
    elif cat == "📂 Ver Todo": query = "SELECT nombre, marca, precio FROM productos"
    else: query = f"SELECT nombre, marca, precio FROM productos WHERE categoria = '{cat}'"
    df = pd.read_sql_query(query, conn).sort_values(by='nombre')
    lista = [f"{row['nombre']} {row['marca']} (${formato_ar(row['precio'])})" for i, row in df.iterrows()]
    return gr.update(choices=lista, value=None)

def formato_num(n): return int(n) if n == int(n) else n
def formatear_cantidad_texto(cantidad, unidad_medida):
    n_limpio = formato_num(cantidad)
    if "kg" in unidad_medida.lower(): return f"{n_limpio} kg"
    palabra = "unidad" if n_limpio == 1 else "unidades"
    return f"{n_limpio} {palabra}"

def generar_html_aviso(stock_restante, promocion, unidad_medida):
    msgs = []
    if stock_restante <= 0: return f"<div style='background-color: #cce5ff; color: #004085; padding: 10px; border-radius: 5px; border: 1px solid #b8daff; text-align: center; font-size: 14px;'>✅ <b>¡Tenés todo el stock disponible en tu carrito!</b><br>{f'🔥 {promocion}' if promocion else ''}</div>"
    elif stock_restante < 10:
        txt_stock = formatear_cantidad_texto(stock_restante, unidad_medida)
        sufijo = "disponible" if stock_restante == 1 else "disponibles"
        msgs.append(f"📦 Quedan {txt_stock} {sufijo}")
    if promocion: msgs.append(f"🔥 ¡OFERTA! {promocion}")
    if msgs: return f"<div style='background-color: #fff3cd; color: #000000 !important; padding: 10px; border-radius: 5px; border: 1px solid #ffeeba; text-align: center; font-size: 14px;'>{'<br>'.join(msgs)}</div>"
    return ""

def actualizar_info_producto(prod_str, carrito):
    if not prod_str: return gr.update(label="Cant"), ""
    nombre = prod_str.split(" ($")[0]
    df = pd.read_sql_query(f"SELECT * FROM productos WHERE nombre || ' ' || marca = '{nombre}'", conn)
    if df.empty: return gr.update(label="Cant"), ""
    row = df.iloc[0]
    label = "Peso (Kg)" if "kg" in row['unidad_medida'] else "Cantidad"
    step = 0.1 if "kg" in row['unidad_medida'] else 1
    cant_en_carrito = 0
    if carrito:
        for item in carrito:
            if item[0] == nombre:
                try: cant_en_carrito = float(str(item[1]).replace(" kg", ""))
                except: pass
                break
    return gr.update(label=label, value=1, step=step), generar_html_aviso(row['stock'] - cant_en_carrito, row['promocion'], row['unidad_medida'])

def calcular_precio_final(precio, cant, promo):
    subtotal = precio * cant
    if not promo: return subtotal, 0, ""
    promo = promo.lower()
    aviso = ""
    if "3x2" in promo or "3 pagas 2" in promo:
        g = int(cant) // 3; s = int(cant) % 3; subtotal = ((g * 2) + s) * precio
        if g > 0: aviso = "🔥 ¡Promo 3x2 Aplicada!"
    elif "2x1" in promo:
        g = int(cant) // 2; s = int(cant) % 2; subtotal = (g + s) * precio
        if g > 0: aviso = "🔥 ¡Promo 2x1 Aplicada!"
    elif "% off" in promo and "caja" not in promo:
        nums = re.findall(r'\d+', promo)
        if nums: subtotal = subtotal * ((100 - int(nums[0])) / 100); aviso = "🔥 Descuento aplicado."
    elif ("caja" in promo or "pack" in promo) and cant >= 6:
        nums = re.findall(r'\d+', promo)
        if nums: subtotal = subtotal * ((100 - int(nums[0])) / 100); aviso = "🔥 Precio Pack."
    elif "2da al 50" in promo:
        p = int(cant) // 2; s = int(cant) % 2; subtotal = (p * precio * 1.5) + (s * precio)
        if p > 0: aviso = "🔥 Promo 2da al 50%."
    return subtotal, subtotal - (precio * cant), aviso

def agregar_al_carrito(prod_str, cant_input, carrito):
    try:
        if not prod_str: return carrito, generar_tabla(carrito), gr.update(), "### 💰 Total: $0,00", ""
        cant = float(cant_input) if cant_input else 0
        if cant == 0: return carrito, generar_tabla(carrito), gr.update(), "### 💰 Total: ...", "⚠️ Cantidad neutra."
        nombre = prod_str.split(" ($")[0]
        df = pd.read_sql_query(f"SELECT * FROM productos WHERE nombre || ' ' || marca = '{nombre}'", conn)
        row = df.iloc[0]
        idx = -1; cant_ant = 0; nuevo_c = list(carrito)
        if carrito:
            for i, item in enumerate(carrito):
                if item[0] == f"{row['nombre']} {row['marca']}":
                    idx = i; cant_ant = float(str(item[1]).replace(" kg", "")); break
        nueva_cant = cant_ant + cant
        if cant > 0 and nueva_cant > row['stock']:
            stock_rest = row['stock'] - cant_ant
            msg = f"<div style='background-color: #f8d7da; color: #721c24; padding: 10px; border-radius: 5px; text-align: center; font-weight: bold;'>⛔ Solo quedan {formato_num(stock_rest)} disponibles.</div>"
            total = sum(i[4] for i in carrito)
            return carrito, generar_tabla(carrito), generar_html_aviso(stock_rest, row['promocion'], row['unidad_medida']), f"### 💰 Total a Pagar: ${formato_ar(total)}", msg
        if nueva_cant <= 0.001:
            if idx != -1: nuevo_c.pop(idx)
            total = sum(i[4] for i in nuevo_c)
            return nuevo_c, generar_tabla(nuevo_c), generar_html_aviso(row['stock'], row['promocion'], row['unidad_medida']), f"### 💰 Total a Pagar: ${formato_ar(total)}", f"<div style='background-color: #f8d7da; color: #721c24; padding: 10px; border-radius: 5px; text-align: center;'>🗑️ {row['nombre']} eliminado.</div>"
        sub, desc, aviso = calcular_precio_final(row['precio'], nueva_cant, row['promocion'])
        cant_fmt = f"{nueva_cant:.3f} kg" if "kg" in row['unidad_medida'] else f"{int(nueva_cant)}"
        item = [f"{row['nombre']} {row['marca']}", cant_fmt, row['precio'], desc if desc < 0 else "-", sub]
        if idx != -1: nuevo_c[idx] = item
        else: nuevo_c.append(item)
        total = sum(i[4] for i in nuevo_c)
        stock_rest_new = row['stock'] - nueva_cant
        desc_ant = 0
        if idx != -1 and carrito:
             try: desc_ant = carrito[idx][3] if isinstance(carrito[idx][3], (int, float)) else 0
             except: pass
        msg_ok = f"<div style='background-color: #d4edda; color: #155724; padding: 10px; border-radius: 5px; text-align: center;'>✅ {aviso}</div>" if desc < desc_ant else f"<div style='background-color: #cce5ff; color: #004085; padding: 10px; border-radius: 5px; text-align: center;'>✔️ Agregado.</div>"
        if cant < 0: msg_ok = f"<div style='background-color: #f8d7da; color: #721c24; padding: 10px; border-radius: 5px; text-align: center;'>📉 Restaste {formatear_cantidad_texto(abs(cant), row['unidad_medida'])}.</div>"
        return nuevo_c, generar_tabla(nuevo_c), generar_html_aviso(stock_rest_new, row['promocion'], row['unidad_medida']), f"### 💰 Total a Pagar: ${formato_ar(total)}", msg_ok
    except Exception as e: return carrito, generar_tabla(carrito), gr.update(), "Error", str(e)

def generar_tabla(lista):
    vis = []
    for i in lista:
        desc = f"${formato_ar(i[3])}" if isinstance(i[3], (int, float)) else "-"
        vis.append([i[0], i[1], f"${formato_ar(i[2])}", desc, f"${formato_ar(i[4])}"])
    return vis

def limpiar(): return [], [], "", "### 💰 Total a Pagar: $0,00", ""

# AUTH & VALIDACIONES
def convertir_fecha(d,m,a):
    try: return True, datetime(int(a), {"Enero":1,"Febrero":2,"Marzo":3,"Abril":4,"Mayo":5,"Junio":6,"Julio":7,"Agosto":8,"Septiembre":9,"Octubre":10,"Noviembre":11,"Diciembre":12}[m], int(d))
    except: return False, None
def registrar_usuario(n, a, d, m, dd, mm, aa, dir, p):
    if not all([n, a, d, m, dd, mm, aa, p]): return "⚠️ Faltan datos"
    if not re.match(r'^\d{7,8}$', d): return "⚠️ DNI inválido"
    ok, f = convertir_fecha(dd, mm, aa)
    if not ok: return "⚠️ Fecha mal"
    if (datetime.now()-f).days/365 < 18: return "⛔ Menor"
    try: cursor.execute("INSERT INTO usuarios VALUES (?,?,?,?,?,?,?)", (d, p, n, a, m, f.strftime('%d/%m/%Y'), dir)); conn.commit(); return "✅ Registrado."
    except: return "❌ DNI existe"
def validar_login(d, p):
    u = cursor.execute("SELECT * FROM usuarios WHERE dni=? AND password=?", (d, p)).fetchone()
    if u: return f"👋 Hola {u[2]}", gr.update(visible=True), gr.update(visible=False)
    else: return "❌ Error", gr.update(visible=False), gr.update(visible=True)
def restaurar_pass(d, m, n):
    if cursor.execute("SELECT * FROM usuarios WHERE dni=? AND mail=?", (d, m)).fetchone(): cursor.execute("UPDATE usuarios SET password=? WHERE dni=?", (n, d)); conn.commit(); return "✅ Listo"
    return "❌ No existe"

with gr.Blocks(theme=gr.themes.Soft()) as app:
    carrito_state = gr.State([])
    with gr.Group() as bloque_acceso:
        with gr.Tab("🔐 Login"):
            dni_in = gr.Textbox(label="DNI"); pass_in = gr.Textbox(label="Clave", type="password"); login_btn = gr.Button("Entrar"); login_out = gr.Textbox(label="Estado")
        with gr.Tab("📝 Registro"):
            with gr.Row():
                n = gr.Textbox(label="Nombre"); a = gr.Textbox(label="Apellido")
            with gr.Row():
                d = gr.Textbox(label="DNI"); m = gr.Textbox(label="Email")
            with gr.Row():
                dd = gr.Dropdown([str(i) for i in range(1,32)], label="Día"); mm = gr.Dropdown(["Enero","Febrero","Marzo","Abril","Mayo","Junio","Julio","Agosto","Septiembre","Octubre","Noviembre","Diciembre"], label="Mes"); aa = gr.Dropdown([str(i) for i in range(1940,2026)], label="Año")
            dir = gr.Textbox(label="Dirección"); p = gr.Textbox(label="Clave", type="password"); reg_btn = gr.Button("Registrar"); reg_out = gr.Textbox(label="Res")
        with gr.Tab("🔄 Recuperar"):
            rd = gr.Textbox(label="DNI"); rm = gr.Textbox(label="Email"); rn = gr.Textbox(label="Nueva Clave", type="password"); r_btn = gr.Button("Cambiar"); r_out = gr.Textbox(label="Est")

    with gr.Group(visible=False) as bloque_principal:
        with gr.Row():
            gr.Markdown("## Tu Supermercado de confianza"); out_btn = gr.Button("Salir", size="sm", variant="stop")
        with gr.Row():
            with gr.Column(scale=1):
                gr.Markdown("### 🤖 Soy tu asistente virtual")
                chat_in = gr.Textbox(label="Consulta"); chat_btn = gr.Button("Enviar"); chat_out = gr.Markdown()
            with gr.Column(scale=2):
                gr.Markdown("### 🛒 Carrito")
                with gr.Row():
                    cat = gr.Dropdown(LISTA_CATEGORIAS_UI, label="Categoría", scale=2); prod = gr.Dropdown([], label="Producto", scale=3); cant = gr.Number(value=1, label="Cant/Peso", precision=1, scale=1)
                add = gr.Button("🔄 Actualizar (Sumar/Restar)", variant="primary"); info = gr.HTML("")
                tabla = gr.Dataframe(headers=["Producto", "Cant", "Precio", "Desc", "Subtotal"], datatype="str", interactive=False)
                avisos = gr.HTML(""); total = gr.Markdown("### 💰 Total: $0,00"); vaciar = gr.Button("Vaciar")

    login_btn.click(validar_login, [dni_in, pass_in], [login_out, bloque_principal, bloque_acceso])
    reg_btn.click(registrar_usuario, [n, a, d, m, dd, mm, aa, dir, p], reg_out)
    r_btn.click(restaurar_pass, [rd, rm, rn], r_out)
    out_btn.click(cerrar, None, [bloque_principal, bloque_acceso, login_out, pass_in, login_out, carrito_state, tabla, info, total, avisos])

    chat_btn.click(responder_chat, chat_in, chat_out); chat_in.submit(responder_chat, chat_in, chat_out)
    cat.change(filtrar_por_categoria, cat, prod)
    prod.change(actualizar_info_producto, [prod, carrito_state], [cant, info])

    add.click(agregar_al_carrito, [prod, cant, carrito_state], [carrito_state, tabla, info, total, avisos])
    vaciar.click(limpiar, None, [carrito_state, tabla, info, total, avisos])

app.launch(share=True)